<a href="https://colab.research.google.com/github/EmmanuelADAM/IntelligenceArtificiellePython/blob/master/TestET-TensorBoard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exemple d'apprentissage du ET avec TensorFlow
## Illustration de l'importance du Bias

|a|b|a et b|
|:-:|:-:|:-:|
|0|0|0|
|0|1|0|
|1|0|0|
|1|1|1|


*Théoriquement, en 1 couche, l'apprentissage du ET par réseau de neurones n'est pas possible.*

En effet, la couche n'est consituée que de 1 neurone (1 sortie), ses entrées sont les valeurs `a` et `b`.
`wa` et `wb` étant les poids affectés à ces valeurs, il faut vérifier : 
 - `f(0)` tend vers 0 --> ok
 - `f(wb)` tend vers 0
 - `f(wa)` tend vers 0
 - `f(wa + wb)` tend vers 1 --> conflit avec les lignes précédentes
 
*Vérifions le...*


---
**Importer les librairies**

In [1]:
#keras : Python Deep Learning library
import tensorflow.keras as keras
#prevision d'utiliser un réseau en couches séquentielles
from tensorflow.keras.models import Sequential
#prevision d'utiliser des couches totalement connectées la précédente
from tensorflow.keras.layers import Dense
#utilisation de la classique librairie pour tableaux, ...
import numpy as np

---
#### Installation du tableau de bord tensorflow pour étudier l'apprentissage
Enregistre des données sur l'évolution des performances dans un fichier log qui sera visualisé par des outils graphiques

In [2]:
##Chargement du tableau de bord tensorflow
%load_ext tensorboard

In [3]:
#besoin du package sur les dates
import datetime

In [4]:
# Efface tous les anciens logs (attention au repertoire)
%rm -rf ./etLogs/

---

## Définir les entrées et sorties attendues


In [5]:
# a et b sont les seules entrées
entrees = np.array([[0,0],
                    [0,1],
                    [1,0],
                    [1,1]], float)

# une seule sortie
sorties = np.array([[0],
                    [0],
                    [0],
                    [1]], float)

---
## 1. Version sans BIAS

### 1.1. Choisir le modèle de réseau 
***ici les couches sont séquentielles***

In [6]:
model = Sequential()

2022-04-25 11:35:08.176177: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### 1.2. Définir l'architecture du réseau
- ici une seule couche constituée de 1 neurone en sortie, 
- de 2 neurones en entrée (pour chaque valeur), 
- utilisation de la sigmoïde comme fonction d'activation

In [7]:
model.add(Dense(1, input_dim=2, use_bias=False, activation='sigmoid'))

---

### 1.3. Compiler le  réseau
Ici, on précise que 
  - l'algo de correction d'erreur est 'Adamax', 
  - l'erreur calculée est la moyenne des valeurs absolues des erreurs commises

In [8]:
model.compile(optimizer='adamax', loss='MSE')

----
#### <font color="red">Ajout de la volonté d'enregistrer dans un log les etapes d'apprentissage</code>


In [9]:
log_dir = "etLogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

2022-04-25 11:35:08.310318: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2022-04-25 11:35:08.310334: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2022-04-25 11:35:08.310403: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.


---

### 1.4. Entraîner le réseau 
- ici on ne le fait pas parler (verbose = 0), 
- et on lance 10000 cycles d'apprentissage
- <font color="red">*Et on indique l'enregistreur de logs (callbacs)*</code>

In [10]:
model.fit(entrees, sorties, verbose=0, epochs=1000, callbacks=[tensorboard_callback])

2022-04-25 11:35:08.374999: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-04-25 11:35:08.625762: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2022-04-25 11:35:08.625783: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2022-04-25 11:35:08.629043: I tensorflow/core/profiler/lib/profiler_session.cc:71] Profiler session collecting data.
2022-04-25 11:35:08.638678: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2022-04-25 11:35:08.653787: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: etLogs/fit/20220425-113508/train/plugins/profile/2022_04_25_11_35_08
2022-04-25 11:35:08.656101: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to etLogs/fit/20220425-113508/train/plugins/profile/2022_04_25_11_35_08/MacBook-Pro-

---

### 1.5. Vérifier le réseau
Etape facultative, en général ***on teste le réseau sur d'autres exemples***. 
- Ici, on n'en a pas. Alors on lui demande de calculer la sortie pour chaque exemple de l'ensemble d'entraînement

In [11]:
entrees

array([[0., 0.],
       [0., 1.],
       [1., 0.],
       [1., 1.]])

In [12]:
predictions = model.predict(entrees)

In [13]:
predictions

array([[0.5       ],
       [0.4572034 ],
       [0.52889764],
       [0.48603192]], dtype=float32)

In [14]:
## petite fonction pour afficher ce qui était attendu et ce qui a ete calculé
def verification(model, entrees, sorties, predictions, bias=False):
    print("verification")
    for i in range(0, len(entrees)):
        print(entrees[i][0], " - ", entrees[i][1], " attendu ", sorties[i], " trouvé ",  predictions[i])

    ws = model.get_weights()
    print("poids pour entree x = " + str(ws[0][0][0]))
    print("poids pour entree y = " + str(ws[0][1][0]))
    if(bias):print("poids pour bias = " + str(ws[1][0]))


In [15]:
verification(model, entrees, sorties, predictions)

verification
0.0  -  0.0  attendu  [0.]  trouvé  [0.5]
0.0  -  1.0  attendu  [0.]  trouvé  [0.4572034]
1.0  -  0.0  attendu  [0.]  trouvé  [0.52889764]
1.0  -  1.0  attendu  [1.]  trouvé  [0.48603192]
poids pour entree x = 0.11571951
poids pour entree y = -0.17160642


---
### 1.6. Affichage des résultats
Ici pas de nécessité de graphique d'évolution de l'erreur.
On affiche les entrées, la sortie attendue, la sortie calculée ainsi que les poids appliquées aux entrées et au signal bias..

In [16]:
loss = model.evaluate(entrees, sorties,verbose=0)
print("perte=",loss)

perte= 0.2507327198982239


**Des erreurs importantes donc**, comme prévu....

---

#### Utilisation du tableau de bord


In [17]:
%tensorboard --logdir etLogs/fit

---
## 2. Version AVEC BIAS

Le tableau est alors

|bias|a|b|a et b|
|:-:|:-:|:-:|:-:|
|1|0|0|0|
|1|0|1|0|
|1|1|0|0|
|1|1|1|1|


*Théoriquement, en 1 couche, l'apprentissage du ET par réseau de neurones est alors possible.*

En effet, la couche n'est consituée que de 1 neurone (1 sortie), ses entrées sont les valeurs `bias`, `a` et `b`.
`wbias`, `wa` et `wb` étant les poids affectés à ces valeurs, il faut vérifier : 
 - `f(bias)` tend vers 0
 - `f(bias + wb)` tend vers 0
 - `f(bias + wa)` tend vers 0
 - `f(bias + wa + wb)` tend vers 1 
 
*Vérifions le...*



----
#### On prépare les fichiers pour le tensorboard (tableau de bord)

In [18]:
# Efface tous les anciens logs (attention au repertoire)
%rm -rf ./etBiasLogs/

---

### 2.1. Définir l'architecture du réseau
- ici une seule couche constituée de 1 neurone en sortie, 
- de 3 neurones en entrée (2 contenant les valeurs + **un Bias** (émettant toujours le signal 1)), 
- utilisation de la sigmoide comme fonction d'activation

In [19]:
model = Sequential()
model.add(Dense(1, input_dim=2, use_bias=True, activation='sigmoid'))

----
#### <font color="red">Ajout de la volonté d'enregistrer dans un log les etapes d'apprentissage</code>


In [20]:
log_dir = "etBiasLogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

2022-04-25 11:35:14.776930: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2022-04-25 11:35:14.776944: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2022-04-25 11:35:14.777005: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.


---

### 2.2 Compiler et entrainer le  réseau 
Ici, on précise que 
  - l'algo de correction d'erreur est 'Adamax', 
  - l'erreur calculée est la moyenne des valeurs absolues des erreurs commises
  - 10000 cycles d'apprentissage demandés
  - <font color="red">*Et on indique l'enregistreur de logs (callbacs)*</code>

In [21]:
model.compile(optimizer='adamax', loss='MSE')

model.fit(entrees, sorties, verbose=0, epochs=10000, callbacks=[tensorboard_callback])

2022-04-25 11:35:15.035928: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2022-04-25 11:35:15.035948: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2022-04-25 11:35:15.041975: I tensorflow/core/profiler/lib/profiler_session.cc:71] Profiler session collecting data.
2022-04-25 11:35:15.042731: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2022-04-25 11:35:15.043967: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: etBiasLogs/fit/20220425-113514/train/plugins/profile/2022_04_25_11_35_15
2022-04-25 11:35:15.044658: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to etBiasLogs/fit/20220425-113514/train/plugins/profile/2022_04_25_11_35_15/MacBook-Pro-de-Emmanuel-2.local.trace.json.gz
2022-04-25 11:35:15.045635: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: etBiasL

---

### 2.2. Vérifier le réseau
Pas d'exemples de validation, on vérifie simplement la correspondance entre sortie attendue et la sortie réelle.

In [22]:
predictions = model.predict(entrees)
verification(model, entrees, sorties, predictions)

verification
0.0  -  0.0  attendu  [0.]  trouvé  [4.3278298e-05]
0.0  -  1.0  attendu  [0.]  trouvé  [0.03388986]
1.0  -  0.0  attendu  [0.]  trouvé  [0.03392008]
1.0  -  1.0  attendu  [1.]  trouvé  [0.9660543]
poids pour entree x = 6.6986203
poids pour entree y = 6.6976967


In [23]:
loss = model.evaluate(entrees, sorties, verbose=0)
print("perte=",loss)

perte= 0.0008628513314761221


**Apprentissage quasi parfait !!!** 
- -> démonstration concrère de l'effet du `Bias` !!

---

## Utilisation du tableau de bord


In [24]:
%tensorboard --logdir etBiasLogs/fit

---
## Test d'entrées non apprises
Le ET conçu par réseau de neurones se comporte maintenant comme un ET en logique floue et est capable d'estimer la valeur de x ET y pour tout x,y dans [0,1].
Exemple pour le test de ET entre deux valeurs *quasi* fausses, une *quasi* vraie ET une *quasi* fausse, et deux valeurs *quasi* vraies : 

In [25]:
# test de 0.2 ET 0.8, 0.1 ET 0.1, 0.8 ET 0.9
tests = np.array([[0.2, 0.3], [0.2, 0.9], [0.8, 0.9]])

predictions = model.predict(tests)
for v in predictions: print(round(v[0],2))

0.0
0.06
0.79
